# マルチスレッド FizzBuzz 問題の詳細解説

## 1. 多角的問題分析

### 競技プログラミング視点
- **同期制御の必要性**: 4つのスレッドが正しい順序で出力する必要がある
- **デッドロック回避**: 適切な待機・通知メカニズムが必須
- **パフォーマンス**: スレッド間通信のオーバーヘッドを最小化

### 業務開発視点
- **スレッドセーフ**: 条件変数やロックを用いた確実な同期
- **保守性**: 明確な状態管理と条件判定
- **型安全性**: Pythonの型ヒントを活用した実装

### Python特有考慮
- **GIL (Global Interpreter Lock)**: CPythonではスレッド並列性に制限があるが、I/O待機では問題なし
- **threading モジュール**: `Condition`を用いた効率的な同期制御
- **型アノテーション**: `Callable`型での関数引数の明示

## 2. アルゴリズム比較表

| アプローチ | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | 標準ライブラリ活用 | CPython最適化 | 備考 |
|---------|---------|---------|--------------|-------|----------------|-------------|-----|
| Condition変数 | O(n) | O(1) | 中 | ★★★ | threading.Condition | 適 | 推奨アプローチ |
| Semaphore | O(n) | O(1) | 高 | ★★☆ | threading.Semaphore | 適 | 複雑になりやすい |
| Event | O(n) | O(1) | 高 | ★☆☆ | threading.Event | 適 | 状態管理が煩雑 |
| Lock + busy wait | O(n²) | O(1) | 低 | ★☆☆ | threading.Lock | 不適 | CPU無駄遣い |

## 3. 採用アルゴリズムと根拠

### 選択: Condition変数による同期制御

**理由:**
1. **効率性**: 条件が満たされるまでスレッドを休止状態にでき、CPU効率的
2. **シンプル性**: 条件判定と通知のパターンが明確
3. **Python標準**: `threading.Condition`が組み込みで提供
4. **デッドロック回避**: 適切な`notify_all()`で全スレッド起床

**動作原理:**
- 各スレッドは自分の担当番号(`current`)が来るまで待機
- 処理後に`current`をインクリメントし、全スレッドに通知
- 次の担当スレッドが処理を引き継ぐ

## 4. 実装パターン

### 業務開発版(型安全・エラーハンドリング重視)

```python
from typing import Callable
from threading import Condition


class FizzBuzz:
    """
    マルチスレッド FizzBuzz 実装
    
    4つのスレッドが協調して1からnまでのFizzBuzz列を出力する。
    Condition変数を用いてスレッド間の同期を実現。
    
    Attributes:
        n: 出力する数値の上限
        current: 現在処理中の番号(1-indexed)
        condition: スレッド同期用のCondition変数
    """
    
    def __init__(self, n: int) -> None:
        """
        FizzBuzzインスタンスを初期化
        
        Args:
            n: 出力する数値の上限(1 <= n <= 50)
            
        Raises:
            ValueError: nが制約を満たさない場合
        """
        if not isinstance(n, int):
            raise TypeError("n must be an integer")
        if not (1 <= n <= 50):
            raise ValueError("n must be between 1 and 50")
            
        self.n: int = n
        self.current: int = 1
        self.condition: Condition = Condition()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        """
        3の倍数(5の倍数を除く)で"fizz"を出力
        
        Args:
            printFizz: "fizz"を出力する関数
        """
        while True:
            with self.condition:
                # 自分の担当番号が来るまで待機
                while self.current <= self.n and not self._is_fizz(self.current):
                    self.condition.wait()
                
                # 終了条件チェック
                if self.current > self.n:
                    break
                
                # 出力処理
                printFizz()
                self.current += 1
                
                # 全スレッドに通知
                self.condition.notify_all()
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        """
        5の倍数(3の倍数を除く)で"buzz"を出力
        
        Args:
            printBuzz: "buzz"を出力する関数
        """
        while True:
            with self.condition:
                while self.current <= self.n and not self._is_buzz(self.current):
                    self.condition.wait()
                
                if self.current > self.n:
                    break
                
                printBuzz()
                self.current += 1
                self.condition.notify_all()
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        """
        3と5の公倍数で"fizzbuzz"を出力
        
        Args:
            printFizzBuzz: "fizzbuzz"を出力する関数
        """
        while True:
            with self.condition:
                while self.current <= self.n and not self._is_fizzbuzz(self.current):
                    self.condition.wait()
                
                if self.current > self.n:
                    break
                
                printFizzBuzz()
                self.current += 1
                self.condition.notify_all()
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        """
        3でも5でも割り切れない数値を出力
        
        Args:
            printNumber: 整数を出力する関数
        """
        while True:
            with self.condition:
                while self.current <= self.n and not self._is_number(self.current):
                    self.condition.wait()
                
                if self.current > self.n:
                    break
                
                printNumber(self.current)
                self.current += 1
                self.condition.notify_all()
    
    def _is_fizzbuzz(self, num: int) -> bool:
        """3と5の両方で割り切れるかチェック"""
        return num % 15 == 0
    
    def _is_fizz(self, num: int) -> bool:
        """3で割り切れるが5では割り切れないかチェック"""
        return num % 3 == 0 and num % 5 != 0
    
    def _is_buzz(self, num: int) -> bool:
        """5で割り切れるが3では割り切れないかチェック"""
        return num % 5 == 0 and num % 3 != 0
    
    def _is_number(self, num: int) -> bool:
        """3でも5でも割り切れないかチェック"""
        return num % 3 != 0 and num % 5 != 0

Analyze Complexity
Runtime 48 ms
Beats 36.97%
Memory 20.19 MB
Beats 5.56%

```

### 競技プログラミング版(性能最優先)

```python
from typing import Callable
from threading import Condition


class FizzBuzz:
    def __init__(self, n: int) -> None:
        self.n = n
        self.current = 1
        self.condition = Condition()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        while True:
            with self.condition:
                while self.current <= self.n and (self.current % 3 != 0 or self.current % 5 == 0):
                    self.condition.wait()
                if self.current > self.n:
                    break
                printFizz()
                self.current += 1
                self.condition.notify_all()
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        while True:
            with self.condition:
                while self.current <= self.n and (self.current % 5 != 0 or self.current % 3 == 0):
                    self.condition.wait()
                if self.current > self.n:
                    break
                printBuzz()
                self.current += 1
                self.condition.notify_all()
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        while True:
            with self.condition:
                while self.current <= self.n and self.current % 15 != 0:
                    self.condition.wait()
                if self.current > self.n:
                    break
                printFizzBuzz()
                self.current += 1
                self.condition.notify_all()
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        while True:
            with self.condition:
                while self.current <= self.n and (self.current % 3 == 0 or self.current % 5 == 0):
                    self.condition.wait()
                if self.current > self.n:
                    break
                printNumber(self.current)
                self.current += 1
                self.condition.notify_all()

Analyze Complexity
Runtime 48 ms
Beats 36.97%
Memory 19.94 MB
Beats 14.10%

```

## 5. Python特有最適化ポイント

### Condition変数の効率的利用
- **`with`文**: 自動的なロック取得・解放でデッドロック防止
- **`notify_all()`**: 全スレッドを起床させ、次の担当が処理を引き継ぐ
- **`wait()`**: スリープ状態でCPU効率的に待機

### 剰余演算の最適化
- **15で割る**: fizzbuzzの判定を最優先(15 = 3 × 5)
- **短絡評価**: `and`/`or`の左から順に評価し、不要な計算を省略

### メモリ効率
- **O(1)空間**: `current`変数のみで状態管理
- **スレッドローカル不要**: Condition変数で十分

## 6. 実装の要点

### 正しい順序保証の仕組み

1. **初期状態**: `current = 1`、全スレッドが待機
2. **条件判定**: 各スレッドは自分の担当番号か確認
3. **処理実行**: 担当スレッドのみが処理を実行
4. **状態更新**: `current`をインクリメント
5. **通知**: `notify_all()`で全スレッドを起床
6. **繰り返し**: 次の番号の担当スレッドが処理

### デッドロック回避

- **`notify_all()`使用**: 特定スレッドだけでなく全スレッドに通知
- **終了条件の共有**: 全スレッドが`current > n`で終了
- **ロック自動解放**: `with`文でContext Manager利用

この実装により、4つのスレッドが協調して正しい順序でFizzBuzz列を出力できます。

# パフォーマンス分析と最適化改善版

## 1. 現状の問題点分析

### ベンチマーク結果の詳細分析

```
業務開発版:
- Runtime: 48ms (Beats 36.97%) ← やや遅い
- Memory: 20.19MB (Beats 5.56%) ← 非常に悪い

競技プログラミング版:
- Runtime: 48ms (Beats 36.97%) ← 変わらず
- Memory: 19.94MB (Beats 14.10%) ← やや改善したが依然として悪い
```

### 根本原因の特定

**メモリ使用量が多い理由:**
1. **`notify_all()`の乱用**: 毎回4スレッド全てを起床させている
2. **無駄なループ**: 条件が合わないスレッドも毎回起床→チェック→再度スリープ
3. **Context Switchingのオーバーヘッド**: 頻繁なスレッド切り替え

**実行時間が遅い理由:**
1. スレッド間の通信コストが高い
2. 不要なスレッドの起床によるCPU浪費

## 2. 最適化戦略

### アプローチ比較

| 最適化手法 | メモリ改善 | 速度改善 | 実装難度 | 備考 |
|---------|---------|---------|---------|------|
| 現在の実装 | × | × | 低 | notify_all()で全スレッド起床 |
| **状態変数 + 個別通知** | ○ | ○ | 中 | 次のスレッドのみ起床 |
| Semaphore連鎖 | △ | △ | 高 | 実装が複雑 |
| Lock + フラグ配列 | ○ | ○ | 中 | シンプルだが保守性低 |

### 採用手法: 状態ベース最小通知アプローチ

**核心的改善点:**
1. **次の担当スレッドのみ起床**: `notify_all()`→条件付き起床
2. **事前計算**: どのスレッドが担当かを事前に判定
3. **無駄な待機削減**: 自分の番でない時は即座にスキップ

## 3. 最適化実装

### 高速・低メモリ版（推奨）

```python
from typing import Callable
from threading import Lock


class FizzBuzz:
    """
    最適化版 FizzBuzz (メモリ・速度重視)
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    最適化ポイント:
    - notify_all()を使わず、必要最小限の通知
    - Conditionの代わりにLock + busy waitの改良版
    - 状態判定を簡潔化
    """
    
    def __init__(self, n: int) -> None:
        self.n: int = n
        self.current: int = 1
        self.lock: Lock = Lock()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        """3の倍数(5の倍数除く)で"fizz"を出力"""
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                    
                # 自分の担当をチェック
                if self.current % 3 == 0 and self.current % 5 != 0:
                    printFizz()
                    self.current += 1
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        """5の倍数(3の倍数除く)で"buzz"を出力"""
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                    
                if self.current % 5 == 0 and self.current % 3 != 0:
                    printBuzz()
                    self.current += 1
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        """15の倍数で"fizzbuzz"を出力"""
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                    
                if self.current % 15 == 0:
                    printFizzBuzz()
                    self.current += 1
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        """3でも5でも割り切れない数値を出力"""
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                    
                if self.current % 3 != 0 and self.current % 5 != 0:
                    printNumber(self.current)
                    self.current += 1

Analyze Complexity
Runtime　1375　ms
Beats　10.90%
Memory　20.00　MB
Beats　14.10%

```

### 超最適化版（競技プログラミング特化）

```python
from typing import Callable
from threading import Lock


class FizzBuzz:
    """
    超最適化版 - インライン展開・剰余演算最小化
    
    最適化技術:
    - 剰余演算の回数削減（15で判定→3と5の個別判定を省略可能なケース）
    - 条件分岐の最適化（頻度の高いケースを先に判定）
    - ロック保持時間の最小化
    """
    
    def __init__(self, n: int) -> None:
        self.n = n
        self.current = 1
        self.lock = Lock()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        while True:
            self.lock.acquire()
            if self.current > self.n:
                self.lock.release()
                break
            
            # インライン条件判定（15の倍数でない3の倍数）
            num = self.current
            if num % 3 == 0 and num % 5 != 0:
                printFizz()
                self.current += 1
            
            self.lock.release()
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        while True:
            self.lock.acquire()
            if self.current > self.n:
                self.lock.release()
                break
            
            num = self.current
            if num % 5 == 0 and num % 3 != 0:
                printBuzz()
                self.current += 1
            
            self.lock.release()
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        while True:
            self.lock.acquire()
            if self.current > self.n:
                self.lock.release()
                break
            
            num = self.current
            # 15で割るのが最速（3と5両方チェック不要）
            if num % 15 == 0:
                printFizzBuzz()
                self.current += 1
            
            self.lock.release()
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        while True:
            self.lock.acquire()
            if self.current > self.n:
                self.lock.release()
                break
            
            num = self.current
            if num % 3 != 0 and num % 5 != 0:
                printNumber(num)
                self.current += 1
            
            self.lock.release()

Analyze Complexity
Runtime　1648　ms
Beats　10.90%
Memory　20.12　MB
Beats　5.56%

```

### 究極最適化版（メモリ最小・速度最大）

```python
from typing import Callable
from threading import Lock


class FizzBuzz:
    """
    究極最適化版
    
    改善点:
    - ビット演算による高速化検討（この問題では効果薄）
    - ロック取得回数の削減
    - 不要な変数コピーを削減
    """
    
    __slots__ = ('n', 'current', 'lock')  # メモリ使用量削減
    
    def __init__(self, n: int) -> None:
        self.n = n
        self.current = 1
        self.lock = Lock()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        lock = self.lock  # ローカル変数化で高速アクセス
        
        while True:
            lock.acquire()
            c = self.current
            
            if c > self.n:
                lock.release()
                break
            
            # 条件式を最適化（短絡評価）
            if c % 3 == 0 and c % 5 != 0:
                printFizz()
                self.current = c + 1
            
            lock.release()
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        lock = self.lock
        
        while True:
            lock.acquire()
            c = self.current
            
            if c > self.n:
                lock.release()
                break
            
            if c % 5 == 0 and c % 3 != 0:
                printBuzz()
                self.current = c + 1
            
            lock.release()
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        lock = self.lock
        
        while True:
            lock.acquire()
            c = self.current
            
            if c > self.n:
                lock.release()
                break
            
            if c % 15 == 0:
                printFizzBuzz()
                self.current = c + 1
            
            lock.release()
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        lock = self.lock
        
        while True:
            lock.acquire()
            c = self.current
            
            if c > self.n:
                lock.release()
                break
            
            if c % 3 != 0 and c % 5 != 0:
                printNumber(c)
                self.current = c + 1
            
            lock.release()

Analyze Complexity
Runtime　1434　ms
Beats　10.90%
Memory　20.28　MB
Beats　5.56%

```

## 4. 最適化の詳細解説

### なぜConditionを捨てたのか？

**Conditionの問題点:**
```python
# 旧実装
with self.condition:
    while self.current <= self.n and not self._is_fizz(self.current):
        self.condition.wait()  # ← スレッドをスリープ
    ...
    self.condition.notify_all()  # ← 全スレッドを起床（無駄）
```

- **notify_all()**: 4スレッド全てを起床させる→3スレッドは再度スリープ
- **待機キュー管理**: Conditionオブジェクトのメモリオーバーヘッド
- **Context Switching**: 頻繁なスレッド切り替えでCPU浪費

**Lockの利点:**
```python
# 新実装
with self.lock:
    if self.current > self.n:
        break
    if self.current % 3 == 0 and self.current % 5 != 0:  # 自分の番だけ処理
        printFizz()
        self.current += 1
    # 通知不要（各スレッドが自律的にチェック）
```

- **自律チェック**: 各スレッドが能動的に確認
- **通知コスト削減**: `notify_all()`が不要
- **シンプル**: メモリオーバーヘッドが少ない

### __slots__ によるメモリ削減

```python
__slots__ = ('n', 'current', 'lock')
```

**効果:**
- 通常のクラス: `__dict__` で属性管理（辞書のオーバーヘッド）
- `__slots__`: 固定サイズの配列で管理（約40%メモリ削減）

### ローカル変数化による高速化

```python
lock = self.lock  # インスタンス変数をローカル化

while True:
    lock.acquire()  # self.lock.acquire() より高速
```

**理由:**
- `self.lock` は属性アクセス（辞書検索相当）
- ローカル変数はスタック上で直接アクセス（高速）

## 5. 期待される改善効果

| バージョン | 予想Runtime | 予想Memory | Beats予想 |
|----------|------------|-----------|----------|
| 元実装 (Condition) | 48ms | 20.19MB | 36% / 5% |
| **高速・低メモリ版 (Lock)** | **35-40ms** | **17-18MB** | **60-70% / 40-50%** |
| 究極最適化版 (__slots__) | **30-35ms** | **16-17MB** | **70-80% / 60-70%** |

### 最も推奨する実装: 高速・低メモリ版

バランスが最も良く、コードも簡潔で保守性が高いため、**高速・低メモリ版**を推奨します。

```python
from typing import Callable
from threading import Lock


class FizzBuzz:
    def __init__(self, n: int) -> None:
        self.n: int = n
        self.current: int = 1
        self.lock: Lock = Lock()
    
    def fizz(self, printFizz: Callable[[], None]) -> None:
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                if self.current % 3 == 0 and self.current % 5 != 0:
                    printFizz()
                    self.current += 1
    
    def buzz(self, printBuzz: Callable[[], None]) -> None:
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                if self.current % 5 == 0 and self.current % 3 != 0:
                    printBuzz()
                    self.current += 1
    
    def fizzbuzz(self, printFizzBuzz: Callable[[], None]) -> None:
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                if self.current % 15 == 0:
                    printFizzBuzz()
                    self.current += 1
    
    def number(self, printNumber: Callable[[int], None]) -> None:
        while True:
            with self.lock:
                if self.current > self.n:
                    break
                if self.current % 3 != 0 and self.current % 5 != 0:
                    printNumber(self.current)
                    self.current += 1
```

この実装でベンチマークを再実行し、結果をご共有いただければ、さらなる最適化のアドバイスが可能です！